<a href="https://colab.research.google.com/github/SlyviaCC/AI/blob/main/Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
from keras.datasets import mnist
import numpy as np
from PIL  import Image           #PIL是python的图像库
import matplotlib.pyplot as plt  #matplotlib.pyplot是一个可以操作图像的函数库
from keras.models import Sequential  #Keras的核心数据结构是model,一种组织网络层的方式，最简单的数据模型是Sequential模型，它是由多个网络层线性
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten  #dense ：全连接层  相当于添加一个层
#Conv2D：二维卷积 14*14
# MaxPooling2D 最大池化
# Flatten '压平'为了实现从卷积完成后压平给到全连接（dense）的过度
'''默认是进行行压平，也可以列压平
from numpy import *
>>> a=array([[1,2],[3,4],[5,6]])
>>> a
array([[1, 2],
    [3, 4],
    [5, 6]])
>>> a.flatten() #默认按行的方向降维
array([1, 2, 3, 4, 5, 6])
>>> a.flatten('F') #按列降维
array([1, 3, 5, 2, 4, 6]) 
>>> a.flatten('A') #按行降维
array([1, 2, 3, 4, 5, 6])
'''

batch_size=32  #BATCH_SIZE:即一次训练所抓取的数据样本数量; BATCH_SIZE的大小影响训练速度和模型优化。同时按照以上代码可知,其大小同样影响每一epoch训练模型次数。
num_classes=10 #看需求，需要多少个类就用多少个类

(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

print(train_images.shape,train_labels.shape)
print(test_images.shape,test_labels.shape)

"""
将数据集中图片展示出来
"""

def show_mnist(train_image,train_labels):
    n = 3
    m = 3
    fig = plt.figure()
    for i in range(n):
        for j in range(m):
            plt.subplot(n,m,i*n+j+1) #subplot是将多个图画到一个平面上的工具。其中,m表示是图排成m行,n表示图排成n列，p是绘图编号
            #plt.subplots_adjust(wspace=0.2, hspace=0.8)
            index = i * n + j #当前图片的标号
            img_array = train_image[index]
            img = Image.fromarray(img_array)  #这个函数的功能是重构图像的数组
            plt.title(train_labels[index])
            plt.imshow(img,cmap='Greys')    #imshow是将矩阵中的每个元素值当作像素值进行显示；（还有说是待处理的图像）
    plt.show()      #原理：plt.imshow()函数负责对图像进行处理，并显示其格式，而plt.show()则是将plt.imshow()处理后的函数显示出来。

img_row,img_col,channel = 28,28,1

mnist_input_shape = (img_row,img_col,1)

#将数据维度进行处理
train_images = train_images.reshape(train_images.shape[0],img_row,img_col,channel)
test_images = test_images.reshape(test_images.shape[0],img_row,img_col,channel)

train_images = train_images.astype("float32")
test_images = test_images.astype("float32")

## 进行归一化处理
# 其功能就是将预处理的数据的数值范围按一定关系“压缩”到一定的范围内。
train_images  /= 255
test_images /= 255

# 将类向量，转化为类矩阵
# 从 5 转换为 0 0 0 0 1 0 0 0 0 0 矩阵
train_labels =tf.keras.utils.to_categorical(train_labels,num_classes)
test_labels =tf.keras.utils.to_categorical(test_labels,num_classes)



"""
构造网络结构
"""
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),
                    activation="relu",
                    input_shape=mnist_input_shape))
                    # kernalsize = 3*3 并没有改变数据维度
model.add(Conv2D(16,kernel_size=(3,3),
                    activation="relu"
                    ))
model.add(MaxPooling2D(pool_size=(2,2)))
                    # 进行数据降维操作
model.add(Flatten())#Flatten层用来将输入“压平”，即把多维的输入一维化，
                    #常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小。
model.add(Dense(32,activation="relu"))
                    #全连接层
model.add(Dense(num_classes,activation='softmax'))

"""
编译网络模型,添加一些超参数
Adadelta 优化是一种随机梯度下降方法，它基于每维度的自适应学习率来解决两个缺点：
1.整个训练过程中学习率的持续衰减。
2.需要手动选择全局学习率。


"""

model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adadelta(),      
                metrics=['accuracy'])
'''实例化是指在面向对象的编程中，把用类创建对象的过程称为实例化。是将一个抽象的概念类，具体到该类实物的过程。
实例化过程中一般由类名 对象名 = new 类名（参数1，参数2...参数n）构成。
函数模板实例化后会生成模板函数，类模板实例化后则会生成模板类。'''
model.fit(train_images,
            train_labels,
            batch_size=batch_size,    
            epochs=5,
            verbose=1,
            validation_data=(test_images,test_labels),
            shuffle=True
            )
'''
batch_size：整数：每次梯度更新的样本数。未指定，默认为32   
epochs:整数:训练模型迭代次数

verbose:日志展示，整数
0:为不在标准输出流输出日志信息
1:显示进度条
2:每个epoch输出一行记录

callbacks
其中的元素是keras.callbacks.Callback的对象。这个list中的回调函数将会在训练过程中的适当时机被调用，参考回调函数

validation_split
浮点数0-1之间
用作验证集的训练数据的比例。
模型将分出一部分不会被训练的验证数据，并将在每一轮结束时评估这些验证数据的误差和任何其他模型指标。
验 证 数 据 是 混 洗 之 前 x 和 y 数 据 的 最 后 一 部 分 样 本 中 

shuffle
布尔值
是否在每轮迭代之前混洗数据

shuffle
布尔值
是否在每轮迭代之前混洗数据

'''
score = model.evaluate(test_images,test_labels,verbose=1)

print('test loss:',score[0])
print('test accuracy:',score[1])

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
Epoch 1/5
1875/1875 [==============================] - 27s 8ms/step - loss: 2.2756 - accuracy: 0.2328 - val_loss: 2.2517 - val_accuracy: 0.2889
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 2.2195 - accuracy: 0.3218 - val_loss: 2.1821 - val_accuracy: 0.3639
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 2.1249 - accuracy: 0.4025 - val_loss: 2.0564 - val_accuracy: 0.4424
Epoch 4/5
1875/1875 [==============================] - 16s 8ms/step - loss: 1.9661 - accuracy: 0.4574 - val_loss: 1.8617 - val_accuracy: 0.4894
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 1.6130 - accuracy: 0.5589
test loss: 1.6129978895187378
test accuracy: 0.558899998664856
